In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models, callbacks
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image

In [2]:
# 將訓練資料拆分成7:3，分別是訓練與測試集，並且取得Info
[train_ds, valid_ds, test_ds], info = tfds.load('colorectal_histology',
                            split=['train[:70%]','train[70%:85%]', 'train[85%:]'],
                            shuffle_files=True,
                            with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/5000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/colorectal_histology/2.0.0.incompleteP8ZDP7/colorectal_histology-train.tfr…

Dataset colorectal_histology downloaded and prepared to /root/tensorflow_datasets/colorectal_histology/2.0.0. Subsequent calls will reuse this data.


In [3]:
# 參數設定
#input_shapes = info.features[info.supervised_keys[0]].shape
input_shapes = (224,224,3)
img_size = input_shapes[:2]
batch_size = 32
epoch = 50

In [4]:
# 預處理函數
def preprocess_data(data):
    image = data['image']
    label = data['label']
    # 將圖片大小調整為指定大小,並正規化像素值至 [0, 1]
    image = tf.image.resize(image, img_size) / 255.0
    return image, label

# 將預處理函數應用到數據集，並將資料分批
train_data = train_ds.map(preprocess_data).batch(batch_size)
valid_data = valid_ds.map(preprocess_data).batch(batch_size)
test_data = test_ds.map(preprocess_data).batch(batch_size)

# EarlyStopping設定
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 建立 DenseNet121 模型
DN121_model = tf.keras.applications.DenseNet121(input_shape=input_shapes, include_top=False, weights='imagenet')
# 凍結 DenseNet121 的權重
DN121_model.trainable = False

29084464/29084464 [==============================] - 0s 0us/step


In [5]:
#DenseNet121 模型 無隱藏層
model_1 = models.Sequential(DN121_model)
model_1.add(layers.GlobalAveragePooling2D())
model_1.add(layers.Dropout(0.5))
model_1.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_1.summary()
# 編譯模型
model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (  (None, 1024)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 8)                 8200      
                                                                 
Total params: 7045704 (26.88 MB)
Trainable params: 8200 (32.03 KB)
Non-trainable params: 7037504 (26.85 MB)
_________________________________________________________________


In [6]:
#DenseNet121 模型 一層隱藏層
model_2 = models.Sequential(DN121_model)
model_2.add(layers.GlobalAveragePooling2D())
model_2.add(layers.Dense(256, activation='relu'))
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_2.summary()
# 編譯模型
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_1  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_1 (Dense)             (None, 256)               262400    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 8)                 2056      
                                                                 
Total params: 7301960 (27.85 MB)
Trainable params: 264456 (1.01 MB)
Non-trainable params: 7037504 (26.85 MB)
___________

In [7]:
#DenseNet121 模型 兩層隱藏層
model_3 = models.Sequential(DN121_model)
model_3.add(layers.GlobalAveragePooling2D())
model_3.add(layers.Dense(256, activation='relu'))
model_3.add(layers.Dense(128, activation='relu'))
model_3.add(layers.Dropout(0.5))
model_3.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_3.summary()
# 編譯模型
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_2  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 256)               262400    
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 8)                 1032      
                                                      

In [8]:
#DenseNet121 模型 三層隱藏層
model_4 = models.Sequential(DN121_model)
model_4.add(layers.GlobalAveragePooling2D())
model_4.add(layers.Dense(256, activation='relu'))
model_4.add(layers.Dense(128, activation='relu'))
model_4.add(layers.Dense(64, activation='relu'))
model_4.add(layers.Dropout(0.5))
model_4.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_4.summary()
# 編譯模型
model_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_3  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_6 (Dense)             (None, 256)               262400    
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                      

In [9]:
#DenseNet121 模型 四層隱藏層
model_5 = models.Sequential(DN121_model)
model_5.add(layers.GlobalAveragePooling2D())
model_5.add(layers.Dense(256, activation='relu'))
model_5.add(layers.Dense(128, activation='relu'))
model_5.add(layers.Dense(64, activation='relu'))
model_5.add(layers.Dense(32, activation='relu'))
model_5.add(layers.Dropout(0.5))
model_5.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_5.summary()
# 編譯模型
model_5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_4  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_10 (Dense)            (None, 256)               262400    
                                                                 
 dense_11 (Dense)            (None, 128)               32896     
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                      

In [10]:
#DenseNet121 模型 無隱藏層
history_1 = model_1.fit(train_data, epochs=epoch, validation_data=valid_data, callbacks=[early_stopping])

Epoch 1/50
110/110 [==============================] - 35s 180ms/step - loss: 1.3015 - accuracy: 0.5383 - val_loss: 0.7031 - val_accuracy: 0.8160
Epoch 2/50
110/110 [==============================] - 12s 106ms/step - loss: 0.7378 - accuracy: 0.7466 - val_loss: 0.5500 - val_accuracy: 0.8587
Epoch 3/50
110/110 [==============================] - 12s 111ms/step - loss: 0.6104 - accuracy: 0.7886 - val_loss: 0.4804 - val_accuracy: 0.8760
Epoch 4/50
110/110 [==============================] - 12s 107ms/step - loss: 0.5482 - accuracy: 0.8149 - val_loss: 0.4451 - val_accuracy: 0.8693
Epoch 5/50
110/110 [==============================] - 12s 108ms/step - loss: 0.5142 - accuracy: 0.8303 - val_loss: 0.4173 - val_accuracy: 0.8840
Epoch 6/50
110/110 [==============================] - 12s 109ms/step - loss: 0.4792 - accuracy: 0.8351 - val_loss: 0.4044 - val_accuracy: 0.8800
Epoch 7/50
110/110 [==============================] - 13s 115ms/step - loss: 0.4567 - accuracy: 0.8414 - val_loss: 0.3836 - val_ac

In [11]:
#DenseNet121 模型 一層隱藏層
history_2 = model_2.fit(train_data, epochs=epoch, validation_data=valid_data, callbacks=[early_stopping])

Epoch 1/50
110/110 [==============================] - 28s 131ms/step - loss: 0.7698 - accuracy: 0.7329 - val_loss: 0.4071 - val_accuracy: 0.8667
Epoch 2/50
110/110 [==============================] - 12s 113ms/step - loss: 0.4312 - accuracy: 0.8543 - val_loss: 0.3641 - val_accuracy: 0.8720
Epoch 3/50
110/110 [==============================] - 12s 112ms/step - loss: 0.3409 - accuracy: 0.8843 - val_loss: 0.3150 - val_accuracy: 0.9000
Epoch 4/50
110/110 [==============================] - 13s 116ms/step - loss: 0.2983 - accuracy: 0.8951 - val_loss: 0.3019 - val_accuracy: 0.9027
Epoch 5/50
110/110 [==============================] - 13s 117ms/step - loss: 0.2567 - accuracy: 0.9089 - val_loss: 0.2850 - val_accuracy: 0.9120
Epoch 6/50
110/110 [==============================] - 12s 111ms/step - loss: 0.2396 - accuracy: 0.9163 - val_loss: 0.2996 - val_accuracy: 0.9040
Epoch 7/50
110/110 [==============================] - 13s 114ms/step - loss: 0.2201 - accuracy: 0.9226 - val_loss: 0.2818 - val_ac

In [12]:
#DenseNet121 模型 兩層隱藏層
history_3 = model_3.fit(train_data, epochs=epoch, validation_data=valid_data, callbacks=[early_stopping])

Epoch 1/50
110/110 [==============================] - 30s 138ms/step - loss: 0.8242 - accuracy: 0.7100 - val_loss: 0.4111 - val_accuracy: 0.8640
Epoch 2/50
110/110 [==============================] - 12s 113ms/step - loss: 0.4408 - accuracy: 0.8520 - val_loss: 0.3699 - val_accuracy: 0.8640
Epoch 3/50
110/110 [==============================] - 13s 113ms/step - loss: 0.3247 - accuracy: 0.8860 - val_loss: 0.3338 - val_accuracy: 0.8867
Epoch 4/50
110/110 [==============================] - 12s 113ms/step - loss: 0.2844 - accuracy: 0.9046 - val_loss: 0.3201 - val_accuracy: 0.8880
Epoch 5/50
110/110 [==============================] - 12s 110ms/step - loss: 0.2336 - accuracy: 0.9191 - val_loss: 0.3395 - val_accuracy: 0.8880
Epoch 6/50
110/110 [==============================] - 13s 117ms/step - loss: 0.2087 - accuracy: 0.9289 - val_loss: 0.2986 - val_accuracy: 0.9000
Epoch 7/50
110/110 [==============================] - 12s 112ms/step - loss: 0.1738 - accuracy: 0.9426 - val_loss: 0.2911 - val_ac

In [13]:
#DenseNet121 模型 三層隱藏層
history_4 = model_4.fit(train_data, epochs=epoch, validation_data=valid_data, callbacks=[early_stopping])

Epoch 1/50
110/110 [==============================] - 22s 136ms/step - loss: 0.9792 - accuracy: 0.6566 - val_loss: 0.4266 - val_accuracy: 0.8720
Epoch 2/50
110/110 [==============================] - 13s 114ms/step - loss: 0.4951 - accuracy: 0.8383 - val_loss: 0.3459 - val_accuracy: 0.8880
Epoch 3/50
110/110 [==============================] - 13s 117ms/step - loss: 0.3712 - accuracy: 0.8734 - val_loss: 0.3330 - val_accuracy: 0.8960
Epoch 4/50
110/110 [==============================] - 13s 115ms/step - loss: 0.3134 - accuracy: 0.9006 - val_loss: 0.3432 - val_accuracy: 0.8987
Epoch 5/50
110/110 [==============================] - 12s 111ms/step - loss: 0.2490 - accuracy: 0.9209 - val_loss: 0.3755 - val_accuracy: 0.8867
Epoch 6/50
110/110 [==============================] - 13s 114ms/step - loss: 0.2108 - accuracy: 0.9334 - val_loss: 0.3711 - val_accuracy: 0.8813


In [14]:
#DenseNet121 模型 四層隱藏層
history_5 = model_5.fit(train_data, epochs=epoch, validation_data=valid_data, callbacks=[early_stopping])

Epoch 1/50
110/110 [==============================] - 22s 137ms/step - loss: 1.1061 - accuracy: 0.5843 - val_loss: 0.4810 - val_accuracy: 0.8467
Epoch 2/50
110/110 [==============================] - 12s 112ms/step - loss: 0.6323 - accuracy: 0.7946 - val_loss: 0.3817 - val_accuracy: 0.8707
Epoch 3/50
110/110 [==============================] - 12s 112ms/step - loss: 0.4923 - accuracy: 0.8346 - val_loss: 0.3503 - val_accuracy: 0.8813
Epoch 4/50
110/110 [==============================] - 13s 114ms/step - loss: 0.3796 - accuracy: 0.8751 - val_loss: 0.3767 - val_accuracy: 0.8720
Epoch 5/50
110/110 [==============================] - 13s 115ms/step - loss: 0.3239 - accuracy: 0.8991 - val_loss: 0.3717 - val_accuracy: 0.8827
Epoch 6/50
110/110 [==============================] - 12s 112ms/step - loss: 0.2959 - accuracy: 0.9109 - val_loss: 0.3307 - val_accuracy: 0.8960
Epoch 7/50
110/110 [==============================] - 12s 111ms/step - loss: 0.2506 - accuracy: 0.9231 - val_loss: 0.4344 - val_ac

In [15]:
model_list = [model_1, model_2, model_3, model_4, model_5]
result = pd.DataFrame(columns=['Accuracy', 'Loss'])

for model in model_list:
  test_loss, test_acc = model.evaluate(test_data)
  result = pd.concat([result, pd.DataFrame({'Accuracy': [test_acc], 'Loss': [test_loss]})], ignore_index=True)

24/24 [==============================] - 2s 88ms/step - loss: 0.2823 - accuracy: 0.9013


In [16]:
result.index = ['無隱藏層', '一層隱藏層', '兩層隱藏層', '三層隱藏層', '四層隱藏層']

result

,Accuracy,Loss
無隱藏層,0.910667,0.268283
一層隱藏層,0.920000,0.205289
兩層隱藏層,0.918667,0.211058
三層隱藏層,0.898667,0.284110
四層隱藏層,0.901333,0.282341
